# Yelp API Restaurant Calls For 2 Coordinates

See calls_by_coords_rating.ipynb for detailed info on code logic

Purpose is to reuse the code from above workbook but hardcode in the 2 coordinates found with >1k restaurant results via the basic api calls passed with geocoordinates & search radius determined by hypotenuse theory.  

    37.798713 -122.405864
    37.755215 -122.405864 	

Whereas all data from the other geocoordinates were retrieved, these 2 coordinates combine to around 3300 missing results.

An additional search for these 2 coordinates is warranted in efforts to obtain this missing info. Yelp API allows for sorting of results before retrieving the first 1k results. Adding this supplemental data to the main data will ensure the inclusion of the top rated restaurants needed for this project while also giving us the maximum possible database of Yelp restaurants in the Bay under these parameters.

Write data to restaurant_data_coord.csv

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys import api_key

### Perform API calls

- Dataframe from grid_coordinates.csv
- Build function that calls and writes restaurant data to new csv, given pair of coordinates
- Run list of coordinates through function

In [2]:
coords_df = pd.DataFrame({"Lat": [37.798713, 37.755215], "Lng": [-122.405864, -122.405864]})
coords_df

,Lat,Lng
0,37.798713,-122.405864
1,37.755215,-122.405864


In [3]:
def get_restaurants(lat, lng, api_key):
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % api_key}
    restaurant_data = []
    yelp_data = []
    count = 0
    
    for offset in range(0, 1000, 50):

    # Set parameters and pass into API calls, radius 8046 meters = 5 miles
        params = {"term":"restaurants", "sort_by":"rating", "latitude":lat, "longitude":lng, "radius":3412, 
              "limit":50, "offset":offset}
        req = requests.get(url, params=params, headers=headers).json()
        
        count += 1
        print(f'Now processing set {count} of max 20')
            
        if len(req["businesses"])>0:
            for business in req['businesses']:
                
                business_dic = {}
                business_dic['Query ID'] = str(lat) + str(lng)
                business_dic['Query Lat'] = lat
                business_dic['Query Lng'] = lng
                
                if 'name' in business:

                    business_dic['Name'] = business['name']
                else:
                    business_dic['Name'] = "NAN"

                business_dic['Category'] = []

                for category in business['categories']:
                    if 'title' in category:
                        business_dic['Category'].append(category['title'])
                if 'coordinates' in business: 
                    if 'latitude' in business['coordinates']:
                        business_dic['Biz Lat'] = business['coordinates']['latitude']
                    else:
                        business_dic['Biz Lat'] = "NAN"
                    if 'longitude' in business['coordinates']:
                        business_dic['Biz Lng'] = business['coordinates']['longitude']
                    else:
                        business_dic['Biz Lng'] = 'NAN'
                if 'location' in business:
                    if 'city' in business['location']:
                        business_dic['Biz City'] = business['location']['city']
                    else:
                        business_dic['Biz City'] = "NAN"
                    if 'address1' in business['location']:
                        business_dic['Biz Address'] = business['location']['address1']
                    else:
                        business_dic['Biz Address'] = "NAN"
                    if 'zip_code' in business['location']:
                        business_dic['Biz Zip'] = business['location']['zip_code']
                    else:
                        business_dic['Biz Zip'] = "NAN"
                if 'price' in business:
                    business_dic['Price'] = business['price']
                else:
                    business_dic['Price'] = "NAN"
                if 'id' in business:
                    business_dic['Yelp ID'] = business['id']
                else:
                    business_dic['Yelp ID'] = "NAN"
                if 'rating' in business:
                    business_dic['Rating'] = business['rating']
                else:
                    business_dic['Rating'] = "NAN"
                if 'review_count' in business:
                    business_dic['Review Count'] = business['review_count']
                else:
                    business_dic['Review Count'] = "NAN"
                yelp_data.append(business_dic)
        else: 
            print("Data not found " +  str(city) + " " + str(offset))
            break

    df = pd.DataFrame(yelp_data)
    df = df[["Query ID", "Query Lat", "Query Lng", "Name", "Category", 
             "Biz Address", "Biz City", "Biz Zip", "Biz Lat", "Biz Lng",
             "Rating", "Review Count", "Price", "Yelp ID"]]
    df.to_csv("Resources/restaurant_data_coords_ratings3.csv")
    
    return req["total"]

In [4]:
# Track # of coordinates processed
count = 0

# List to track total restaurants found in coordinate
totals_count = []

print("LOG HISTORY OF API CALLS:")
print("---------------------------")

# Loop thru list of coordinates
lat_coords = [lat for lat in coords_df["Lat"]]
lng_coords = [lng for lng in coords_df["Lng"]]

for index in range(len(lat_coords)):
                   
    #  Call get_restaurants fn and append to total_count list
    totals_count.append(get_restaurants(lat_coords[index], lng_coords[index] , api_key))
    
    # Print log history
    count += 1
    rem = len(lat_coords) - count
    print("-----------------------------------------")
    if count == len(lat_coords):
        print("Full list of coordinates processed!")
    elif count == len(lat_coords) - 1:
        print("Now getting results for final coordinates. Almost there!")
        print("-----------------------------------------")
    else:
        print(f"Data retrieval for coordinates {count} complete")
        print(f"Getting results for next coordinates.. there are {rem} left")
        print("-----------------------------------------")

coords_df["Total # of Restaurants"] = totals_count

LOG HISTORY OF API CALLS:
---------------------------
Now processing set 1 of max 20
Now processing set 2 of max 20
Now processing set 3 of max 20
Now processing set 4 of max 20
Now processing set 5 of max 20
Now processing set 6 of max 20
Now processing set 7 of max 20
Now processing set 8 of max 20
Now processing set 9 of max 20
Now processing set 10 of max 20
Now processing set 11 of max 20
Now processing set 12 of max 20
Now processing set 13 of max 20
Now processing set 14 of max 20
Now processing set 15 of max 20
Now processing set 16 of max 20
Now processing set 17 of max 20
Now processing set 18 of max 20
Now processing set 19 of max 20
Now processing set 20 of max 20
-----------------------------------------
Now getting results for final coordinates. Almost there!
-----------------------------------------
Now processing set 1 of max 20
Now processing set 2 of max 20
Now processing set 3 of max 20
Now processing set 4 of max 20
Now processing set 5 of max 20
Now processing set 

In [5]:
# Show coords_df with updated total # of restaurants
# Should still match previous results. ie 2700 & 2600 respectively
coords_df.head()

,Lat,Lng,Total # of Restaurants
0,37.798713,-122.405864,2700
1,37.755215,-122.405864,2600
